# Image Slice inferencing
Testing image slice inferenceing for microservice setup

# 1) Imports and mount

In [1]:
%load_ext autoreload
%autoreload 2

# This sets up the appropriate logging and path configs
from notebook_setup import * 

In [2]:
import torch
from torchsummary import summary
import matplotlib.pyplot as plt

from monai.losses import DiceLoss
from src.features.build_features import train_transform, val_transform, test_transform

from src.visualization.visualize import view_slice

from monai.data import Dataset, DataLoader
from src.enums import DataDict

from src.settings.config import get_app_settings
from src.utils import load_yaml

## 2) Import Settings

In [3]:
settings = get_app_settings()
model_config = load_yaml("model_configs.yaml")

## 3) Do Encode and Decoding Testing

In [4]:
import base64
import io
import nibabel as nib

def encode_nifti_to_base64(file_path):
    # Load NIfTI file
    img = nib.load(file_path)
    
    # Convert NIfTI image to binary data
    nifti_binary = img.to_bytes()
    
    # Encode binary data to base64
    base64_encoded = base64.b64encode(nifti_binary)
    
    return base64_encoded

def decode_base64_to_nifti(base64_encoded, output_file_path):
    # Decode base64 to binary data
    nifti_binary = base64.b64decode(base64_encoded)
    
    # Save binary data to a new NIfTI file
    with open(output_file_path, 'wb') as f:
        f.write(nifti_binary)

    return nifti_binary

def load_nifti_from_bytes(nifti_bytes):
    # Create a BytesIO object to simulate a file-like object from bytes
    bytes_io = io.BytesIO(nifti_bytes)

    # Load NIfTI image from BytesIO object
    nifti_img = nib.fileutils.read_zt_byte_strings(bytes_io)

    return nifti_img

In [5]:

# Example usage:
file_path = "data/interim/training/Singapore/57/Image/Flair/brain_57-slice010_z.nii.gz"
encoded_data = encode_nifti_to_base64(file_path)

# Save or transmit the base64 encoded data as needed
# ...
print(type(encoded_data))
# Example of decoding and saving the NIfTI file back
# output_file_path = 'path/to/your/output/file.nii.gz'
# decode_base64_to_nifti(encoded_data, output_file_path)

<class 'bytes'>


In [31]:
bytes_io = io.BytesIO(encoded_data)
output_file_path = 'data/temp/file_temp.nii.gz'
decoded = decode_base64_to_nifti(encoded_data, output_file_path)
print(type(decoded))

<class 'bytes'>


In [ ]:
img_a = klass.from_bytes(bstr)
img_b = klass.from_bytes(img_a.to_bytes())

np.allclose(img_a.get_fdata(), img_b.get_fdata())
np.allclose(img_a.affine, img_b.affine)

In [ ]:
from nibabel.spatialimages import SpatialImage